In [238]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import os, sys
import json
import pickle

In [239]:
from tqdm import tqdm
tqdm.pandas()

### Чтение данных

In [240]:
file_path = '/data/share/project01/gender_age_dataset.txt'

In [241]:
nrows = None #10
df = pd.read_csv(file_path, sep='\t', nrows=nrows)

In [242]:
df.head()

,gender,age,uid,user_json
0,F,18-24,d50192e5-c44e-4ae8-ae7a-7cfe67c8b777,"{""visits"": [{""url"": ""http://zebra-zoya.ru/2000..."
1,M,25-34,d502331d-621e-4721-ada2-5d30b2c3801f,"{""visits"": [{""url"": ""http://sweetrading.ru/?p=..."
2,F,25-34,d50237ea-747e-48a2-ba46-d08e71dddfdb,"{""visits"": [{""url"": ""http://ru.oriflame.com/pr..."
3,F,25-34,d502f29f-d57a-46bf-8703-1cb5f8dcdf03,"{""visits"": [{""url"": ""http://translate-tattoo.r..."
4,M,>=55,d503c3b2-a0c2-4f47-bb27-065058c73008,"{""visits"": [{""url"": ""https://mail.rambler.ru/#..."


### Чистка данных

In [243]:
# убеждаемся, что целевые переменные либо одновременно заполнены, либо одновременно пусты
assert len(df.loc[(df['age'] == '-') & (df['gender'] != '-')]) == 0
assert len(df.loc[(df['gender'] == '-') & (df['age'] != '-')]) == 0

In [244]:
# отделяем данные для обучения
df = df.loc[(df['age'] != '-') & (df['gender'] != '-')]

### Генерация признаков

In [277]:
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

In [278]:
from urllib.parse import urlparse
from urllib.request import urlretrieve, unquote

In [279]:
class ExtractDomainTransformer(BaseEstimator, TransformerMixin):
    """Добавляет столбец со списком доменов."""        
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X['user_json'].progress_apply(lambda x: [url2domain(el['url']) for el in json.loads(x)['visits']])
    
    @staticmethod
    def url2domain(url):
        url = re.sub('(http(s)*://)+', 'http://', url)
        parsed_url = urlparse(unquote(url.strip()))
        if parsed_url.scheme not in ['http','https']: return None
        netloc = re.search("(?:www\.)?(.*)", parsed_url.netloc).group(1)
        if netloc is not None: return netloc.strip()
        return None

In [280]:
from sklearn.feature_extraction.text import CountVectorizer

In [281]:
class ToFloatTransformer(BaseEstimator, TransformerMixin):
    """Приводит элементы матрицы признаков к типу float64."""
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, y=None):
        return X.astype(np.float64)

### Кодирование целевой переменной

In [282]:
from sklearn.preprocessing import OrdinalEncoder

In [283]:
enc = OrdinalEncoder()

In [284]:
y = enc.fit_transform(df[['gender', 'age']])

In [285]:
enc.categories_

[array(['F', 'M'], dtype=object),
 array(['18-24', '25-34', '35-44', '45-54', '>=55'], dtype=object)]

In [286]:
y

array([[0., 0.],
       [1., 1.],
       [0., 1.],
       ...,
       [1., 1.],
       [1., 0.],
       [1., 1.]])

In [300]:
X = df

### Обучение

In [287]:
from sklearn.model_selection import train_test_split

In [288]:
# разбиваем данные на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [289]:
class TwoOutputClassifier(BaseEstimator, ClassifierMixin):  
    """Классификатор, предсказывающий две целевые переменные 
       с использованием базовых классификаторов."""
    def __init__(self, first_classifier, second_classifier):
        self.first_classifier = first_classifier
        self.second_classifier = second_classifier

    def fit(self, X, y):
        self.first_classifier.fit(X, y[:,0])
        self.second_classifier.fit(X, y[:,1])
        return self
        
    def predict(self, X):
        return np.stack([self.first_classifier.predict(X), self.second_classifier.predict(X)], axis=1)

In [290]:
from lightgbm.sklearn import LGBMClassifier

### Построение Pipeline

In [291]:
from sklearn.pipeline import Pipeline

In [305]:
def identity(x):
    return x

In [306]:
pipeline = Pipeline([('extract_domain', ExtractDomainTransformer()),
                     ('count_domain', CountVectorizer(analyzer=identity)),
                     ('to_float', ToFloatTransformer()),
                     ('clf', TwoOutputClassifier(LGBMClassifier(verbose=2), LGBMClassifier(verbose=2)))],
                    verbose=True)

In [293]:
pipeline.fit(X_train, y_train)


 30%|██▉       | 8113/27103 [00:19<00:41, 452.89it/s]


 58%|█████▊    | 15795/27103 [00:36<00:26, 419.99it/s]


 86%|████████▌ | 23375/27103 [00:53<00:07, 491.98it/s]


100%|██████████| 27103/27103 [01:01<00:00, 439.14it/s]

Pipeline(memory=None,
         steps=[('extract_domain', ExtractDomainTransformer()),
                ('count_domain',
                 CountVectorizer(analyzer=<function <lambda> at 0x7f26249b0ea0>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None,...
                                     second_classifier=LGBMClassifier(boosting_type='gbdt',
                                                                      class_weight=None,
                                                                      colsample_bytree=1.0,
                                                                      learning_rate=0.1,
                       

### Оценка предсказательной способности

In [294]:
# предсказываем целевую переменную
y_pred = pipeline.predict(X_test)


 89%|████████▉ | 8053/9035 [00:17<00:01, 576.60it/s]


100%|██████████| 9035/9035 [00:18<00:00, 476.24it/s]

In [295]:
y_pred

array([[0., 1.],
       [1., 1.],
       [1., 2.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 1.]])

In [296]:
# считаем долю полностью совпадающих строк матриц y_true и y_pred
def full_accuracy(y_true, y_pred):
    return sum(np.logical_and(y_test[:,0] == y_pred[:,0], y_test[:,1] == y_pred[:,1])) / len(y_test)

In [297]:
full_accuracy(y_test, y_pred)

0.30282235749861647

### Сохранение модели

In [310]:
pipeline.fit(X, y)


 28%|██▊       | 10021/36138 [00:18<00:44, 587.46it/s]


 48%|████▊     | 17385/36138 [00:39<00:35, 524.29it/s]


 71%|███████   | 25496/36138 [00:58<00:12, 838.29it/s]


100%|██████████| 36138/36138 [01:16<00:00, 471.28it/s]

[Pipeline] .... (step 1 of 4) Processing extract_domain, total= 1.3min
[Pipeline] ...... (step 2 of 4) Processing count_domain, total=   2.3s
[Pipeline] .......... (step 3 of 4) Processing to_float, total=   0.0s
[Pipeline] ............... (step 4 of 4) Processing clf, total=  33.1s


Pipeline(memory=None,
         steps=[('extract_domain', ExtractDomainTransformer()),
                ('count_domain',
                 CountVectorizer(analyzer=<function identity at 0x7f1d1bfdd7b8>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None,...
                                     second_classifier=LGBMClassifier(boosting_type='gbdt',
                                                                      class_weight=None,
                                                                      colsample_bytree=1.0,
                                                                      learning_rate=0.1,
                       

In [311]:
import pickle

In [313]:
pickle.dump([pipeline, enc], open('model.pickle', 'wb'))